In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os, sys, subprocess, importlib
from pathlib import Path

# Disable trackers
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"
os.environ["OMP_NUM_THREADS"] = "1"

# (Optional but recommended) remove crashy optional deps if present
subprocess.run([sys.executable, "-m", "pip", "-q", "uninstall", "-y", "tensorboard", "ray", "wandb"], check=False)

# Ensure ultralytics exists (do NOT install deps again)
subprocess.run([sys.executable, "-m", "pip", "-q", "install", "--no-deps", "ultralytics==8.2.0"], check=True)

WORK="/kaggle/working"
if WORK not in sys.path:
    sys.path.insert(0, WORK)
os.environ["PYTHONPATH"] = WORK + ":" + os.environ.get("PYTHONPATH","")

# Make sitecustomize so DDP subprocess ranks also inherit safe loading
Path(f"{WORK}/sitecustomize.py").write_text(r'''
import os
os.environ.setdefault("WANDB_DISABLED", "true")
os.environ.setdefault("WANDB_MODE", "disabled")
os.environ.setdefault("WANDB_SILENT", "true")
os.environ.setdefault("OMP_NUM_THREADS", "1")

try:
    import torch
    try:
        from ultralytics.nn.tasks import DetectionModel
        torch.serialization.add_safe_globals([DetectionModel])
    except Exception:
        pass

    if not getattr(torch, "_weights_only_off", False):
        _orig = torch.load
        def patched(*args, **kwargs):
            kwargs.setdefault("weights_only", False)
            try:
                return _orig(*args, **kwargs)
            except TypeError:
                kwargs.pop("weights_only", None)
                return _orig(*args, **kwargs)
        torch.load = patched
        torch._weights_only_off = True
except Exception:
    pass
''')

sys.modules.pop("sitecustomize", None)
importlib.invalidate_caches()
import sitecustomize  # noqa

import torch, ultralytics
print("✅ torch:", torch.__version__, "cuda:", torch.cuda.is_available(), "gpus:", torch.cuda.device_count())
print("✅ ultralytics:", ultralytics.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 13.4 MB/s eta 0:00:00
✅ torch: 2.8.0+cu126 cuda: True gpus: 2
✅ ultralytics: 8.2.0


In [4]:
from pathlib import Path

# Find best.pt from your input dataset
best_candidates = list(Path("/kaggle/input").glob("**/baseline_640_2gpu/weights/best.pt"))
assert best_candidates, "❌ best.pt not found. Ensure session-1-yolo-11v is added as Input."
best_pt = best_candidates[0]
print("✅ best_pt:", best_pt)

# Find dataset root
data_candidates = list(Path("/kaggle/input").glob("**/randomized_dataset"))
assert data_candidates, "❌ randomized_dataset not found. Ensure crackathon-data is added as Input."
DATA_ROOT = data_candidates[0]
print("✅ DATA_ROOT:", DATA_ROOT)

# Write YAML
yaml_path = Path("/kaggle/working/rdd.yaml")
yaml_path.write_text(f"""
path: {DATA_ROOT}
train: train/images
val: val/images
test: test/images
names:
  0: longitudinal_crack
  1: transverse_crack
  2: alligator_crack
  3: other_corruption
  4: pothole
""".strip()+"\n")
print("✅ YAML:", yaml_path)


✅ best_pt: /kaggle/input/seven-hours/baseline_640_2gpu/weights/best.pt
✅ DATA_ROOT: /kaggle/input/crackathon-data/randomized_dataset
✅ YAML: /kaggle/working/rdd.yaml


In [ ]:
import torch
from ultralytics import YOLO

device_val = 0 if torch.cuda.is_available() else "cpu"
m = YOLO(str(best_pt))
res = m.val(data=str(yaml_path), imgsz=640, device=device_val, verbose=False)
print("✅ val mAP50:", float(res.box.map50))
print("✅ val mAP50-95:", float(res.box.map))


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"   # ✅ allow both GPUs
os.environ["NCCL_IB_DISABLE"] = "1"
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
!nvidia-smi
!nvidia-smi --query-compute-apps=pid,process_name,used_memory --format=csv,noheader


In [6]:
import os, signal, subprocess

me = os.getpid()
q = subprocess.check_output(
    "nvidia-smi --query-compute-apps=pid,process_name --format=csv,noheader",
    shell=True
).decode().strip().splitlines()

killed = []
for line in q:
    pid_s, name = [x.strip() for x in line.split(",")]
    pid = int(pid_s)
    if pid != me and ("python" in name.lower()):
        try:
            os.kill(pid, signal.SIGKILL)
            killed.append(pid)
        except Exception as e:
            print("Could not kill", pid, e)

print("✅ killed:", killed)


✅ killed: []


In [7]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()
print("✅ cleared")
!nvidia-smi


✅ cleared
Mon Jan 12 11:02:29 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------------------------

In [ ]:
import time, torch
from ultralytics import YOLO

DEVICE = "0,1" if torch.cuda.is_available() and torch.cuda.device_count() >= 2 else 0
BATCH = 32  # safest for 2xT4 at 640

TARGET = 0.50
TIME_LIMIT_SEC = int(4.0 * 3600)
START = time.time()

def get_map5095(trainer):
    m = getattr(trainer, "metrics", {})
    if not isinstance(m, dict): return None
    v = m.get("metrics/mAP50-95(B)", None)
    if v is None: v = m.get("metrics/mAP50-95", None)
    return float(v) if v is not None else None

def stop_cb(trainer):
    if getattr(trainer, "rank", 0) in (-1, 0):
        v = get_map5095(trainer)
        if v is not None:
            print(f"\n[EPOCH {trainer.epoch+1}] val mAP50-95={v:.4f}")
            if v >= TARGET:
                print("✅ Baseline reached, stopping Stage A.")
                trainer.stop = True
                return
    if time.time() - START > TIME_LIMIT_SEC:
        print("⏱️ Stage A time limit reached.")
        trainer.stop = True

model = YOLO(str(best_pt))   # your carried weight
model.callbacks.setdefault("on_fit_epoch_end", [])
model.callbacks["on_fit_epoch_end"].append(stop_cb)

model.train(
    data=str(yaml_path),
    imgsz=640,
    epochs=400,
    batch=BATCH,
    device=DEVICE,
    workers=2,
    cache=False,
    amp=True,
    project="runs",
    name="stageA_640",
    patience=0,

    optimizer="SGD",
    lr0=0.005,
    lrf=0.01,
    cos_lr=True,

    mosaic=0.10,
    mixup=0.0,
    translate=0.10,
    scale=0.30,
    fliplr=0.5,
)


New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.0 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
                                                           CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/seven-hours/baseline_640_2gpu/weights/best.pt, data=/kaggle/working/rdd.yaml, epochs=400, time=None, patience=0, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0,1, workers=2, project=runs, name=stageA_640, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, str

100%|██████████| 755k/755k [00:00<00:00, 17.2MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           


  7                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  8                  -1  3   4461568  ultralytics.nn.modules.block.C2f             [512, 512, 3, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  3   4723712  ultralytics.nn.modules.block.C2f             [1024, 512, 3]                
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  3   1247744  ultralytics.nn.modules.block.C2f             [768,

100%|██████████| 6.23M/6.23M [00:00<00:00, 69.2MB/s]
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/checks.py:640: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):


...

  0%|          | 0/825 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py:368: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.amp):
     16/400        13G      1.378      1.168      1.449         11        640: 100%|██████████| 825/825 [12:41<00:00,  1.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 188/188 [02:06<00:00,  1.49it/s]


                   all       6000      10443      0.633      0.585      0.611      0.326

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/825 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py:368: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.amp):
     17/400      13.2G      1.374      1.148      1.442         18        640: 100%|██████████| 825/825 [12:41<00:00,  1.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 188/188 [02:06<00:00,  1.49it/s]


                   all       6000      10443      0.646      0.581      0.611      0.328

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/825 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py:368: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.amp):
     18/400      13.4G      1.354      1.113      1.428         21        640:  18%|█▊        | 150/825 [02:13<10:08,  1.11it/s]